In [1]:

import torch
import torch.optim as optim
import torch.nn as nn
from parse_data import get_data, get_modified_values, get_binary_values, make_data_scalar
import numpy as np
import random
from data_gen import Datagen
from recognition import Recognition
from generator import Generator
from evaluation import evaluate_model, bin_plot
from time_recognition import TimeRecognition

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device=None
print("Using device: ", device)

import torch
print(torch.__version__)
 

Using device:  cuda
1.12.0+cu116


In [2]:
gen = Datagen(device)

x, y, x_1 = gen.get_generated_data(seq_len=2)

print("x", x[0])
print("y", y[0])
print("x_1", x_1[0])

x tensor([[0.],
        [0.]], device='cuda:0')
y tensor([1.], device='cuda:0')
x_1 tensor([[0.],
        [1.]], device='cuda:0')


In [3]:
import random

# Hyperparameters
sequence_length = [2*i for i in range(4,16)] # 2-20 increments of two
hidden_layers = [1,2] # 1 and 2
hidden_1 = [2**i for i in range(2,7)] # 2^4 to 2^9
hidden_2 =[2**i for i in range(5,10)] # 2^2 to 2^5
variance = [0.001, 0.01, 0.005, 0.05]
lr = [0.001, 0.01, 0.1, 0.005] # stop at 0.005
data_probability = [i/5 for i in range(1,6)]
regularization = [1/i for i in range(1,10)]
for i in range(3):
    regularization.append(0)

epochs = 500
optimizer = [optim.Adam, optim.SGD]

options = []

for seq_len in sequence_length:
    for layers in hidden_layers:
        for h1 in hidden_1:
            for h2 in hidden_2:
                for l in lr:
                    for v in variance:
                        for p in data_probability:
                            for r in regularization:
                                entry = {}
                                entry["seq_len"] = seq_len
                                entry["layers"] = layers
                                entry["latent"] = h1
                                entry["hidden"] = h2
                                entry["l"] = l
                                entry["variance"] = v
                                entry["data_prob"] = p
                                entry["regularization"] = r
                                options.append(entry)
                
                                         
random.shuffle(options)    


In [ ]:


import torch.utils.data as data
from itertools import chain
import torch.nn.functional as F

def loss(x, x_hat, mean, R, s, x_1,reg,  device=None, seq_len=1):
    
    mse = nn.MSELoss().to(device)
    l = F.binary_cross_entropy(x_hat, x, reduction='sum')
    amount = mean[0].size()[0]*mean[0].size()[1]
    for m, r in zip(mean, R):
        
        C = r @ r.transpose(-2,-1) + 1e-6
        det = C.det() + 1e-6 
        l += 0.5 * torch.sum(m.pow(2).sum(-1) 
                             + C.diagonal(dim1=-2,dim2=-1).sum(-1)
                            -det.log()  -1)/amount

    count = len(s)*2
    for a, b in zip(s, x_1):
        l += reg*mse(a[0], b[0])/count
        l += reg*mse(a[1], b[1])/count
    
    #print(l, F.binary_cross_entropy(x_hat, x, reduction='sum'))
    return l 

best_model = None
best_score = 10000000000000000
batch_size = 10
best_history= [0,0,0,0,0,0]
for entry in options:

    x_d, y_d, x_1_d = gen.get_generated_data(entry["seq_len"], entry["variance"], entry["data_prob"])
    x_t, y_t, x_t_1 = gen.get_true_data(entry["seq_len"])
    x_val, y_val, x_val_1 = gen.get_test_data(entry["seq_len"])


    model_t = TimeRecognition(input_dim=x_d[0].size()[1],
                              hidden_size=entry["hidden"],
                              seq_len=entry["seq_len"],
                              layers=entry["layers"],
                             device=device)

    model_g = Generator(hidden_size=entry["hidden"],
                        latent_dim=entry["latent"],
                        output_dim=y_d[0].size()[0],
                        layers=entry["layers"],
                        seq_len=entry["seq_len"],
                        device=device)
    model_r = Recognition(input_dim=x_d[0].size()[1],
                          latent_dim=entry["latent"],
                          layers=entry["layers"],
                          device=device)

    loader = data.DataLoader(data.TensorDataset(x_d, y_d, x_1_d), batch_size=batch_size, shuffle=True)
    optimizer = optim.Adam(chain(model_r.parameters(), model_g.parameters(), model_t.parameters()), lr=0.01)
    #optimizer = optim.Adam(model_r.parameters())
    history = []
    bce = nn.BCELoss().to(device)
    for e in range(epochs):
        model_g.train()
        model_r.train()
        model_t.train()


        for x, y, x_1 in loader:

            x.to(device)
            y.to(device)
            if x.size()[0] < batch_size:
                continue
            if random.random() < 0.5:
                continue

            t = model_t(x)
            t_1 = model_t(x_1)
            model_g.make_internal_state()
            rec = model_r(x_1)
            model_g.set_xi(rec[-1])
            model_g.set_internal_state(t)
            b, s = model_g()

            l = loss(y, b, rec[0], rec[1], s, t_1, entry["regularization"], device, entry["seq_len"])
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
                        

        
        if e % 10 != 0:
            continue
        
        count = 0
        sum_loss = [0, 0]
        for j in range(2):
            for x, y, x_1 in loader:
                model_g.eval()
                model_t.eval()
                model_r.eval()
                model_g.make_internal_state()
                model_g.make_xi()
                with torch.no_grad():
                    model_g.make_internal_state()
                    rec = model_r(x_1)
                    t = model_t(x)
                    t_1 = model_t(x_1)
                    model_g.set_internal_state(t)
                    model_g.set_xi(rec[-1])
                    b,s = model_g()
                    l = loss(y, b, rec[0], rec[1],s,t_1,entry["regularization"], device, entry["seq_len"])
                    res = []
                    
                    sum_loss[j] += l.item()
                    count += 1
                    
        
        
        sum_loss[0] /= count
     
        
        history.append([e, sum_loss[0], sum_loss[1]])
        print(history[-1])

        if len(history) > 15:
            #if no real improvements are being done stop the training. 
            # but keep doing the training if the results without correctly feeding values get better
            if abs(history[-15][1] - history[-1][1]) < 0.0001:
                break
        
    
    if history[-1][1] < best_score:
        print("New best model:\nNew loss: ", history[-1], "\nOld loss:", best_history[-1], "\nHistory:" , history[-10:])
        best_model = model_g
        best_history = history
        best_score = history[-1][1]
        best_config = entry
        with torch.no_grad():
            evaluate_model(best_model,model_r, model_t, x_t, y_t, x_t_1,x_val,y_val, x_val_1, entry)
    else:
        with torch.no_grad():
            evaluate_model(model_g,model_r, model_t, x_t, y_t, x_t_1,x_val,y_val, x_val_1, entry)
        print("Old model still stands:\nCurrent loss: ", history[-1], "\nBest loss:", best_history[-1])
    

[10, 7.183002580239318, 11020.6673746109]
[20, 7.238397690397019, 11085.505493164062]
[30, 7.219948619215047, 11017.67986869812]
[40, 7.221480355250306, 11082.172514915466]
[50, 7.3234477703004845, 11196.203893661499]
[60, 7.240130959229432, 11097.616790771484]
[70, 7.309230597772424, 11183.62596321106]
[80, 7.218655926751717, 11083.661992073059]
[90, 7.210942463214964, 11062.433305740356]
[100, 7.194042856948494, 11048.292337417603]
[110, 7.255224343690798, 11142.379161834717]
[120, 7.205919951122673, 11033.069692611694]
[130, 7.210334184586846, 11050.64178943634]
[140, 7.190562615506643, 11030.767473220825]
[150, 7.201941186391965, 11058.502060890198]
[160, 7.200841938546681, 11040.048259735107]
[170, 7.187624763882192, 11044.901119232178]
[180, 7.220659988667261, 11075.382726669312]
[190, 7.174888125287646, 11023.61337852478]
[200, 7.204359663995688, 11037.897086143494]
[210, 7.23074166395023, 11054.731225013733]
[220, 7.224494919141963, 11070.69748878479]
[230, 7.197062110776378, 1

Featurizing p:   0%|          | 0/1000 [00:00<?, ?it/s]

Featurizing q:   0%|          | 0/3000 [00:00<?, ?it/s]

Featurizing p:   0%|          | 0/1000 [00:00<?, ?it/s]

Featurizing q:   0%|          | 0/3000 [00:00<?, ?it/s]

...

Old model still stands:
Current loss:  [490, 13.070631265017756, 20028.19465637207] 
Best loss: [490, 3.433227788063627, 5257.182981014252]
[0, 7.2281988800038866, 11069.042422294617]
[10, 7.296327894723758, 11172.568161964417]
[20, 7.24367381698469, 11085.388676643372]
[30, 7.232542816092389, 11075.796612739563]
[40, 7.2210150063816, 11053.273871421814]
[50, 7.339766492420326, 11247.643494606018]
[60, 7.216008122222542, 11052.559129714966]
[70, 7.229036391559531, 11100.294623374939]
[80, 7.266934467669255, 11121.331694602966]
[90, 7.2013199273350965, 11000.240265846252]
[100, 7.227924704862948, 11056.10896396637]
[110, 7.195872515362175, 11037.959086418152]
[120, 7.254515327921736, 11118.891551017761]
[130, 7.556383086869362, 11584.726097106934]
[140, 7.297148060238393, 11178.423092842102]
[150, 7.237752941818835, 11092.271032333374]
[160, 7.21236638985477, 11039.9593334198]
[170, 7.261471201794577, 11093.417952537537]
[180, 7.216116129573892, 11052.30354309082]
[190, 7.26195337878194

Featurizing p:   0%|          | 0/1000 [00:00<?, ?it/s]

Featurizing q:   0%|          | 0/3000 [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[100, 33.67898583972423, 51601.20234680176]
[110, 33.502887835390574, 51319.5551071167]
[120, 33.464323123503604, 51299.46531677246]
[130, 33.55226404673126, 51416.02634429932]
[140, 33.484492762593, 51307.63957977295]
[150, 33.521741568264076, 51344.49035644531]
[160, 33.5290400303375, 51327.16030883789]
[170, 33.484469236966525, 51304.66120147705]
[180, 33.51798505260179, 51349.04504394531]
[190, 33.44905003679639, 51233.41590118408]
[200, 33.47063831996669, 51298.23156738281]
[210, 33.531719110653235, 51353.78197479248]
[220, 33.736507694652744, 51681.36962890625]
[230, 33.5739697936932, 51474.42967224121]
[240, 33.56911430657688, 51440.28953552246]


In [ ]:
a = torch.zeros(10,5,1)

In [ ]:
a[:,-1,:].size()

In [ ]:
b = torch.zeros(30,1)
c = torch.zeros(1,1)


In [ ]:
torch.cat((b[1:],c))

In [ ]:
23